In [1]:
!git clone https://github.com/eg4000/SKU110K_CVPR19.git

fatal: destination path 'SKU110K_CVPR19' already exists and is not an empty directory.


In [1]:
import os
os.chdir('SKU110K_CVPR19')

In [2]:
# import keras_retinanet
from object_detector_retinanet.keras_retinanet import models
from object_detector_retinanet.keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from object_detector_retinanet.keras_retinanet.utils.visualization import draw_box, draw_caption
from object_detector_retinanet.keras_retinanet.utils.colors import label_color

Using TensorFlow backend.


In [3]:
# import for EM Merger and viz
from object_detector_retinanet.keras_retinanet.utils import EmMerger
from object_detector_retinanet.utils import create_folder, root_dir


# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

In [4]:
model_path = '/home/vicky/Downloads/iou_resnet50_csv_06.h5'
model = models.load_model(model_path, backbone_name='resnet50',convert=True)

/home/vicky/anaconda3/envs/sku/lib/python3.6/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
# image_path="/home/vicky/Documents/SKU110K/images/IMG-20201008-WA0009.jpg"

In [12]:
def get_prediction(image_path,output_path):
    image = read_image_bgr(image_path)

    # for filtering predictions based on score (objectness/confidence)
    threshold = 0.3

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # Run inference
    boxes, hard_scores, labels, soft_scores = model.predict_on_batch(np.expand_dims(image, axis=0))
    hard_score_rate=.3
    max_detections = 9999

    soft_scores = np.squeeze(soft_scores, axis=-1)
    soft_scores = hard_score_rate * hard_scores + (1 - hard_score_rate) * soft_scores

    # correct boxes for image scale
    boxes /= scale

    # select indices which have a score above the threshold
    indices = np.where(hard_scores[0, :] > threshold)[0]

    # select those scores
    scores = soft_scores[0][indices]
    hard_scores = hard_scores[0][indices]

    # find the order with which to sort the scores
    scores_sort = np.argsort(-scores)[:max_detections]

    # select detections
    image_boxes = boxes[0, indices[scores_sort], :]
    image_scores = scores[scores_sort]
    image_hard_scores = hard_scores[scores_sort]
    image_labels = labels[0, indices[scores_sort]]
    image_detections = np.concatenate(
        [image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)
    results = np.concatenate(
        [image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_hard_scores, axis=1),
         np.expand_dims(image_labels, axis=1)], axis=1)
    filtered_data = EmMerger.merge_detections(image_path, results)
    filtered_boxes = []
    filtered_scores = []
    filtered_labels = []
    
    csv_data_lst = []
    csv_data_lst.append(['image_id', 'x1', 'y1', 'x2', 'y2', 'confidence', 'hard_score'])
    
    for ind, detection in filtered_data.iterrows():
        box = np.asarray([detection['x1'], detection['y1'], detection['x2'], detection['y2']])
        filtered_boxes.append(box)
        filtered_scores.append(detection['confidence'])
        filtered_labels.append('{0:.2f}'.format(detection['hard_score']))
        row = [image_path, detection['x1'], detection['y1'], detection['x2'], detection['y2'],
               detection['confidence'], detection['hard_score']]
        csv_data_lst.append(row)
    print(csv_data_lst)
    labels_to_names = {0: 'object'}
    for box, score, label in zip(filtered_boxes, filtered_scores, filtered_labels):
        # scores are sorted so we can break
        if score < threshold:
            break

        color = [31, 0, 255]#label_color(label) ## BUG HERE LABELS ARE FLOATS SO COLOR IS HARDCODED 

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[0], score)
        draw_caption(draw, b, caption)
    detected_img =cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)

#     cv2.imwrite(output_path,detected_img)

In [13]:
import os

In [14]:
# plt.figure(figsize=(20, 20))
# plt.axis('off')
# plt.imshow(draw)
# plt.show()

In [15]:
path='/home/vicky/Downloads/HUL SAMPLES'
output='/home/vicky/Documents/SKU110K_Output/'

In [17]:
# os.makedirs(output)

In [18]:
for file in os.listdir(path):
    input_path=os.path.join(path,file)
    output_path=os.path.join(output,file)
    print(file)
    get_prediction(input_path,output_path)
    break
    

Hair_GT (7).jpeg
[['image_id', 'x1', 'y1', 'x2', 'y2', 'confidence', 'hard_score'], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 891, 725, 950, 889, 0.7985255718231201, 0.7158021926879883], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 847, 730, 886, 860, 0.7296908497810364, 0.5288412570953369], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 353, 727, 424, 895, 0.7822343707084656, 0.648598313331604], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 678, 714, 731, 907, 0.6939334869384766, 0.41353335976600647], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 429, 718, 505, 895, 0.799734890460968, 0.7152411341667175], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 263, 707, 326, 894, 0.6777764558792114, 0.38792943954467773], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 162, 707, 257, 912, 0.8432467579841614, 0.8275237083435059], ['/home/vicky/Downloads/HUL SAMPLES/Hair_GT (7).jpeg', 552, 703, 612, 889, 0.8336702585220337, 0.8248674

/home/vicky/Notebooks/SKU110K_CVPR19/object_detector_retinanet/keras_retinanet/utils/EmMerger.py:81: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  best_detection_id = original_detections.avg_score.argmax()
/home/vicky/Notebooks/SKU110K_CVPR19/object_detector_retinanet/keras_retinanet/utils/Boxes.py:148: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  image_boxes = image_data.as_matrix(BOX_CONSTANTS)


In [30]:
# draw.shape

In [ ]:
bin